# The Qmod Workshop - Introduction

The Classiq platform features a high-level quantum modeling language called Qmod. Qmod is compiled into concrete gate-level implementation using a powerful synthesis engine that optimizes and adapts the implementation to different target hardware/simulation environments.

In this workshop, we will learn how to write quantum models using Qmod. We will be using the Python embedding of Qmod, available as part of the Classiq Python SDK. We will learn basic concepts in the Qmod language, such as functions, operators, quantum variables, and quantum types. We will develop useful building blocks and small algorithms.

The [QMOD language reference](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/) covers these concepts more systematically and includes more examples.

This workshop consists of step-by-step exercises. It is structured as follows:

- Part 1: Language Fundamentals - Exercises 1-5
- Part 2: Higher-Level Concepts - Exercises 6-10
- Part 3: Execution Flows - Exercises 11, 12

The introduction and Part 1 are included in this notebook. Part 2 and 3 are each in its own separate notebook. For each exercise you will find the solution to the exercises at the bottom of the same notebook.

### Preparations

Make sure you have a Python version of 3.8 through 3.11 installed. Unfortunately, Classiq is not yet supported with Python 3.12.

Install Classiq’s Python SDK by following the instructions on this page: [Getting Started - Classiq](https://docs.classiq.io/latest/getting-started/).

### Python Qmod Exercises - General Instructions

In order to synthesize and execute your Qmod code, you should:
1. Make sure you define a `main` function that calls functions you create.
2. Use `create_model` by running `qmod = create_model(main)` to construct a representation of your model.
3. You can synthesize the model (using `qprog = synthesize(qmod)`) to obtain an implementation - a quantum program.
4. You can then visualize the quantum program (`show(qprog)`) or execute it (using `execute(qprog)`. See: [Execution - Classiq](https://docs.classiq.io/latest/user-guide/platform/executor/#full-example)). You can also execute it with the IDE after visualizing the circuit.


### Exercise 0: From Model to Execution

The following model defines a function that applies X and H gates on a single qubit, and subsequently calls it:

In [1]:
from classiq import *


# Define a quantum function using the @qfunc decorator
@qfunc
def foo(q: QBit) -> None:
    X(target=q)
    H(target=q)


# Define a main function
@qfunc
def main(res: Output[QBit]) -> None:
    allocate(1, res)
    foo(q=res)

Create a model from it, and synthesize, visualize, and execute it.

Use the General Instructions above to do so.


In [2]:
from classiq import *

# Your code here:

quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

In [3]:
show(quantum_program)                          # let's see the quantum program in the online IDE

Opening: https://platform.classiq.io/circuit/ce259560-876c-40a5-8974-f7add71592d7?version=0.41.2


In [4]:
job = execute(quantum_program)                 # let's execute the quantum program

# job.open_in_ide()  # to view in IDE

results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we see the result state ('res') and how many counts for each one (in front fo the state)

[{'res': 0.0}: 1029, {'res': 1.0}: 1019]


In Qmod `QBit` is the simplest quantum type, and in this example, `q` is a quantum variable of type `QBit`. Quantum variables abstract away the mapping of quantum objects to qubits in the actual circuit.

See also [Quantum Variables](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/quantum-variables/).

We will discuss other quantum types during the workshop.


# The Qmod Workshop - Part 1: Language Fundamentals

Follow exercises 1 through 5 for the first session of the workshop.

## Exercise 1 - Bell Pair

Create a function that takes two single-qubit (`QBit`) quantum arguments and prepares the bell state on them ([Bell state](https://en.wikipedia.org/wiki/Bell_state)) by applying `H` on one variable and then using it as the control of a `CX` function with the second variable as the target.
Create a main function that uses this function and has two single-qubit outputs, initialize them to the |0> state (using the `allocate` function), and apply your function to them.

See also [Functions](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/functions#syntax)


In [5]:
from classiq import *

# Your code here:

@qfunc
def prepare_my_bell_state(x:QBit, y:QBit) -> None:
    H(x)
    CX(x, y)

@qfunc
def main(x: Output[QBit], y: Output[QBit]) -> None:
    allocate(1, x)
    allocate(1, y)
    prepare_my_bell_state(x, y)




qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/87a1d782-0826-4337-b59a-4ef7616cf3fc?version=0.41.2


In [6]:
job = execute(qprog)
results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we can see that we obtained the states '00' and '11' with half the probability for each one

[{'x': 0.0, 'y': 0.0}: 1042, {'x': 1.0, 'y': 1.0}: 1006]


Use qubit array subscript (the syntax - _variable_ **[** _index-expression_ **]**) to change the function from subsection 1 to receive a single quantum variable, a qubit array (`QArray`) of size 2.
Change your main function to declare a single output (also an array of size 2).


In [7]:
from classiq import *

# Your code here:

@qfunc
def prepare_my_bell_state_2(reg: QArray[QBit]) -> None:
    H(reg[0])
    CX(reg[0], reg[1])

@qfunc
def main(reg: Output[QArray]) -> None:
    allocate(2, reg)
    prepare_my_bell_state_2(reg)




qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/f0a77a16-a5ba-4e0e-941a-d0d2171efcfd?version=0.41.2


In [8]:
job = execute(qprog)
results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we can see that we obtained the states '00' (=0) and '11' (=3) with half the probability for each one

[{'reg': 3.0}: 1047, {'reg': 0.0}: 1001]


## Exercise 2 - Repeat

Use the built-in `repeat` operator to create your own Hadamard transform function (call it `my_hadamard_transform`). The Hadamard transform function is a function that takes as argument a qubit array of an unspecified size and applies `H` to each of its qubit.

See also [Classical repeat](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/classical-control-flow/#classical-repeat).

Set your main function to have a quantum array output of unspecified size, allocate 10 qubits, and then apply your Hadamard transform function.


In [9]:
from classiq import *

# Your code here:

@qfunc
def my_hadamard_transform(reg: QArray[QBit]) -> None:
    repeat(reg.len, lambda i: H(reg[i]))

@qfunc
def main(reg: Output[QArray]) -> None: 
    allocate(10, reg)
    my_hadamard_transform(reg)




qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/120d57a5-c745-4e4b-828e-165606094fbd?version=0.41.2


In [10]:
job = execute(qprog)
results = job.result()
parsed_counts = results[0].value.parsed_counts

sorted_states = [int(sampled_state.state['reg']) for sampled_state in parsed_counts]
sorted_states.sort()
print(sorted_states)                   # we can see all the possible states for 10 qubits (from 0 to 2^{10} - 1 = 1023)

[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 200, 201, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 

### Note: Quantum Variable Capture
The `repeat` operator invokes a statement block multiple times. The statement block is specified using a Python callable, typically a lambda expression. Inside the block you can refer to variables declared in the outer function scope.
This concept is called `quantum variable capture`, equivalent to [capture](https://en.wikipedia.org/wiki/Closure_(computer_programming)) in classical languages.

See also [Capturing context variables and parameters](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/operators/#capturing-context-variables-and-parameters).

### Exercise 3 - Power
Raising a quantum operation to a power appears in many known algorithms, for examples, in Grover search and Quantum Phase Estimation.
For most operations, it simply means repeating the same circuit multiple times.

Sometimes, however, power can be simplified, thus saving computational resources.
The most trivial example is a quantum operation expressed as a single explicit unitary matrix (i.e., all n*n matrix terms are given) -  raising the operation can be done by raising the matrix to that power via classical programming.

See also [Power operator](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/#syntax).

Use the following code to generate a 2-qubit (real) unitary matrix:

In [11]:
from typing import List

import numpy as np

from classiq import *

rng = np.random.default_rng(seed=0)
random_matrix = rng.random((4, 4))
qr_unitary, _ = np.linalg.qr(random_matrix)

unitary_matrix = QConstant("unitary_matrix", List[List[float]], qr_unitary.tolist())

In order to reuse some classical value we can define a `QConstant` to store that value.

1. Create a model that applies `unitary_matrix` on a 2 qubit variable.
2. Create another model that applies `unitary_matrix` raised to power 3 on a 2 qubit variable.
3. Compare the gate count via the Classiq’s IDE in both cases.

Note - the signature of function `unitary` is:

```python
def unitary(
    elements: CArray[CArray[CReal]],
    target: QArray[QBit],
) -> None:
    pass
```

In [12]:
from classiq import *

# Your code here:

@qfunc
def apply_unitary_matrix(reg: QArray[QBit], exponent: CInt) -> None:
    power(exponent, lambda: unitary(unitary_matrix, reg))


In [13]:
# Your code here:

@qfunc
def main(reg: Output[QArray]) -> None:        # apply the unitary matrix once
    allocate(2, reg)
    apply_unitary_matrix(reg, 1)




qmod1 = create_model(main)
qprog1 = synthesize(qmod1)
show(qprog1)

# from the IDE:
#   Program info
#       Depth: 7
#       Width: 2
#   Gate count
#       U : 8
#       CX : 3

Opening: https://platform.classiq.io/circuit/878c7001-8a90-4de1-a940-59365437e376?version=0.41.2


In [14]:
# Your code here:

@qfunc
def main(reg: Output[QArray]) -> None:        # apply the unitary matrix thrice
    allocate(2, reg)
    apply_unitary_matrix(reg, 3)



qmod3 = create_model(main)
qprog3 = synthesize(qmod3)
show(qprog3)

# from the IDE:
#   Program info
#       Depth: 7
#       Width: 2
#   Gate count
#       U : 8
#       CX : 3

Opening: https://platform.classiq.io/circuit/ca21082c-49a9-4536-831e-54281264dad3?version=0.41.2


## Exercise 4 - User-defined Operators
Create a function that applies a given single-qubit operation to all qubits in its quantum argument (Call your function `my_apply_to_all`). Such a function is also called an operator, i.e. a function that one of its arguments is another function (its operand).

See also [Operators](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/operators/).

Follow these guidelines:
1. Your function should declare a quantum argument of type qubit array. It should also declare an argument of a function type with a single qubit argument.
2. The body should apply the operand to all qubits in the argument.

When you're done, re-implement `my_hadamard_transform` from exercise 2 using this function instead of `repeat`.
Use the same main function from exercise 2.

In [15]:
from classiq import *

# Your code here:


@qfunc
def my_apply_to_all(my_operand: QCallable[QBit], reg: QArray[QBit]) -> None:
    # apply_to_all(gate_operand=my_operand, target=reg)    # this is another option
    repeat(reg.len, lambda i: my_operand(reg[i]))

@qfunc
def my_hadamard_transform(reg: QArray[QBit]) -> None:
    my_apply_to_all(H, reg)

@qfunc
def main(reg: Output[QArray]) -> None: 
    allocate(10, reg)
    my_hadamard_transform(reg)

    


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/6bceb651-fc52-49e7-86d4-eee1f284bef4?version=0.41.2


In [16]:
job = execute(qprog)
results = job.result()
parsed_counts = results[0].value.parsed_counts

sorted_states = [int(sampled_state.state['reg']) for sampled_state in parsed_counts]
sorted_states.sort()
print(sorted_states)                   # we can see all the possible states for 10 qubits (from 0 to 2^{10} - 1 = 1023)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 65, 67, 68, 69, 70, 71, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 107, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 141, 143, 144, 145, 146, 147, 149, 150, 151, 152, 154, 155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 170, 171, 172, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 206, 207, 208, 210, 211, 212, 213, 214, 215, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 254, 255,

# Exercise 5 -  Quantum Conditionals

### Exercise 5a - Control Operator
Use the built-in `control` operator to create a function that receives two single qubit variables and uses one of the variables to control an RY gate with a `pi/2` angle acting on the other variable (without using the `CRY` function).

See also [Quantum operators](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/#syntax).


In [17]:
from classiq import *

# Your code here:

from math import pi

@qfunc
def my_cotrolled_ry(x:QBit, y:QBit) -> None:
    control(x, lambda: RY(pi/2, y))

@qfunc
def main(x: Output[QBit], y: Output[QBit]) -> None:
    allocate(1, x)
    allocate(1, y)
    X(x)                    # so that the implemented Controlled-RY does apply RY to 'y'
    my_cotrolled_ry(x, y)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/88b602ea-79f9-4e44-8323-b47712e662c4?version=0.41.2


In [18]:
job = execute(qprog)                           # let's execute the quantum program
results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we can see that we obtained the states '10' and '11' with half the probability for each one
                                               # 'x' is always 1 because we apply the X gate and 'y' is in superposition (0 and 1)

[{'x': 1.0, 'y': 1.0}: 1047, {'x': 1.0, 'y': 0.0}: 1001]


### Exercise 5b - Control ("Quantum If")
The `control` operator is the conditional application of some operation, with the condition being that all control qubits are in the state |1>. This notion is generalized in QMOD to other control states, where the condition is specified as a comparison between a quantum numeric variable and a numeric value, similar to a classical `if` statement. Quantum numeric variables are declared with class `QNum`.

See also [Numeric types](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/quantum-types/#syntax).

In QMOD this generalization is available as a native statement - control.

See also [control](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/).

1. Declare a `QNum` output argument using `Output[QNum]` and name it `x`.
2. Use the `prepare_int` function to initialize it to `9`. Note that you don't need to specify the `QNum` attributes - size, sign, and fraction digits, as they are inferred at the point of initialization.
3. Execute the circuit and observe the results.
4. Declare another output argument of type `QBit` and perform a `control` such that under the condition that `x` is 9, the qubit is flipped. Execute the circuit and observe the results. Repeat for a different condition.

In [19]:
from classiq import *

# Your code here:

@qfunc
def main(x:Output[QNum]):
    prepare_int(9, x)



qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/c89d8573-cf97-4546-8871-0c4a8d584699?version=0.41.2


In [20]:
job = execute(qprog)                           # let's execute the quantum program
results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we observe the prepared state: 9 (=1001) with all probability

[{'x': 9.0}: 2048]


In [21]:
from classiq import *

# Your code here:

@qfunc
def my_controlled_x(x: QNum, target: QBit) -> None:
    control(x == 9, lambda: X(target))

@qfunc
def main(x:Output[QNum], res: Output[QBit]):
    allocate(1, res)
    prepare_int(9, x)
    my_controlled_x(x, res)



qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/cc68c0cf-a56f-4d03-91db-0955166d76ff?version=0.41.2


In [22]:
job = execute(qprog)                           # let's execute the quantum program
results = job.result()
parsed_counts = results[0].value.parsed_counts
print(parsed_counts)                           # we observe the prepared state: 9 (=1001) and the state 1 for 'res' with all probability
                                               # in this way we see the state: 25 (=11001), in the histogram when running in the IDE

[{'x': 9.0, 'res': 1.0}: 2048]
